In [1]:
### For first-time users, the StructuralIdentifiability package can be installed by the following command

#using Pkg
#Pkg.add("StructuralIdentifiability")


In [2]:
using StructuralIdentifiability

Singular.jl, based on
                     SINGULAR                               /
 A Computer Algebra System for Polynomial Computations     /  Singular.jl: 0.5.8
                                                         0<   Singular   : 4.2.0p1
 by: W. Decker, G.-M. Greuel, G. Pfister, H. Schoenemann   \
FB Mathematik der Universitaet, D-67653 Kaiserslautern      \
     


In [3]:
#if you want to supress the Info blocks
using Logging
Logging.disable_logging(Logging.Info)

LogLevel(1)

In [4]:
#v known, no parameters fixed
ode_br = @ODEmodel(

    50=50.0, 
    # g_Na=0.04, 
    # g_Nac=3e-05, 
    # g_s=0.0009, 
    IstimAmplitude=0.5,
    IstimEnd=50000.0, 
    IstimPeriod=1000.0, 
    IstimPulseDuration=1.0,
    IstimStart=10.0, 
    C=0.01

    # Expressions for the Sodium current component
    i_Na = (g_Nac + g_Na * (m * m * m) * h * j) * (-E_Na + V),

    # Expressions for the m gate component
    alpha_m = (-47 - V) / (-1 + 0.009095277101695816 * exp(-0.1 * V)),
    beta_m = 0.7095526727489909 *  exp(-0.056 * V),
    m'(t) = (1 - m) * alpha_m - beta_m * m,

    # Expressions for the h gate component
    alpha_h = 5.497962438709065e-10 *  exp(-0.25 * V),
    beta_h = 1.7 / (1 + 0.1580253208896478 *  exp(-0.082 * V)),
    h'(t) = (1 - h) * alpha_h - beta_h * h,

    # Expressions for the j gate component
    alpha_j = (1.8690473007222892e-10 *  exp(-0.25 * V) / (1 + 1.6788275299956603e-07 *  exp(-0.2 * V))  ),
    beta_j = 0.3 / (1 + 0.040762203978366204 *  exp(-0.1 * V)),
    j'(t) = (1 - j) * alpha_j - beta_j * j,

    # Expressions for the Slow inward current component
    E_s = -82.3 - 13.0287 *  log(0.001 * Cai),
    i_s = g_s * (-E_s + V) * d * f,
    Cai'(t) = 7.000000000000001e-06 - 0.07 * Cai - 0.01 * i_s,

    # Expressions for the d gate component
    alpha_d = (
        0.095
        * math.exp(1 / 20 - V / 100)
        / (1 + 1.4332881385696572 * math.exp(-0.07199424046076314 * V))
    ),
    beta_d = 0.07 * math.exp(-44 / 59 - V / 59) / (1 + math.exp(11 / 5 + V / 20)),
    d'(t) = (1 - d) * alpha_d - beta_d * d,

    # Expressions for the f gate component
    alpha_f = (
        0.012
        * math.exp(-28 / 125 - V / 125)
        / (1 + 66.5465065250986 * math.exp(0.14992503748125938 * V))
    ),
    beta_f = 0.0065 * math.exp(-3 / 5 - V / 50) / (1 + math.exp(-6 - V / 5)),
    f'(t) = (1 - f) * alpha_f - beta_f * f,

    # Expressions for the Time dependent outward current component
    i_x1 = (
        0.0019727757115328517
        * (-1 + 21.75840239619708 * math.exp(0.04 * V))
        * math.exp(-0.04 * V)
        * x1
    ),

    # Expressions for the X1 gate component
    alpha_x1 = (
        0.031158410986342627
        * math.exp(0.08264462809917356 * V)
        / (1 + 17.41170806332765 * math.exp(0.05714285714285714 * V))
    ),
    beta_x1 = (
        0.0003916464405623223
        * math.exp(-0.05998800239952009 * V)
        / (1 + math.exp(-4 / 5 - V / 25))
    ),
    x1'(t) = (1 - x1) * alpha_x1 - beta_x1 * x1

    # Expressions for the Time independent outward current component
    i_K1 = 0.0035 * (4.6000000000000005 + 0.2 * V) / (
        1 - 0.39851904108451414 * math.exp(-0.04 * V)
    ) + 0.0035 * (-4 + 119.85640018958804 * math.exp(0.04 * V)) / (
        8.331137487687693 * math.exp(0.04 * V) + 69.4078518387552 * math.exp(0.08 * V)
    ),

    # Expressions for the Stimulus protocol component
    Istim = (
        IstimAmplitude
        if t - IstimStart - IstimPeriod * math.floor((t - IstimStart) / IstimPeriod)
        <= IstimPulseDuration
        and t <= IstimEnd
        and t >= IstimStart
        else 0
    ),
    # IstimAmplitude=0.5, IstimEnd=50000.0, IstimPeriod=1000.0,
    # IstimPulseDuration=1.0, IstimStart=10.0

    # Expressions for the Membrane component
    V'(t) = (-i_K1 - i_Na - i_s - i_x1 + Istim) / C,

    y1(t) = m(t),
    y2(t) = h(t),
    y3(t) = j(t),
    y4(t) = Cai(t),
    y5(t) = d(t),
    y6(t) = f(t),
    y7(t) = x1(t),
    y8(t) = V(t),
)

v'(t) = I_ext - w(t) - v(t)^3 + v(t)
w'(t) = (-w(t)*b - a + v(t))//tau
y(t) = v(t)


In [ ]:
#v known, no parameters fixed
ode_br = @ODEmodel(
    50=50, 
    # g_Na=0.04, 
    # g_Nac=3e-05, 
    # g_s=0.0009, 
    IstimAmplitude=0.5,
    IstimEnd=50000.0, 
    IstimPeriod=1000.0, 
    IstimPulseDuration=1.0,
    IstimStart=10.0, 
    C=0.01,

    # Expressions for the Sodium current component
    i_Na = (g_Nac + g_Na * (m * m * m) * h * j) * (-E_Na + V),

    # Expressions for the m gate component
    alpha_m = (-47 - V) / (-1 + 0.009095277101695816 * exp(-0.1 * V)),
    beta_m = 0.7095526727489909 *  exp(-0.056 * V),
    m'(t) = (1 - m) * alpha_m - beta_m * m,

    # Expressions for the h gate component
    alpha_h = 5.497962438709065e-10 *  exp(-0.25 * V),
    beta_h = 1.7 / (1 + 0.1580253208896478 *  exp(-0.082 * V)),
    h'(t) = (1 - h) * alpha_h - beta_h * h,

    # Expressions for the j gate component
    alpha_j = (1.8690473007222892e-10 *  exp(-0.25 * V) / (1 + 1.6788275299956603e-07 *  exp(-0.2 * V))  ),
    beta_j = 0.3 / (1 + 0.040762203978366204 *  exp(-0.1 * V)),
    j'(t) = (1 - j) * alpha_j - beta_j * j,

    # Expressions for the Slow inward current component
    E_s = -82.3 - 13.0287 *  log(0.001 * Cai),
    i_s = g_s * (-E_s + V) * d * f,
    Cai'(t) = 7.000000000000001e-06 - 0.07 * Cai - 0.01 * i_s,

    # Expressions for the d gate component
    alpha_d = (
        0.095
        * math.exp(1 / 20 - V / 100)
        / (1 + 1.4332881385696572 * math.exp(-0.07199424046076314 * V))
    ),
    beta_d = 0.07 * math.exp(-44 / 59 - V / 59) / (1 + math.exp(11 / 5 + V / 20)),
    d'(t) = (1 - d) * alpha_d - beta_d * d,

    # Expressions for the f gate component
    alpha_f = (
        0.012
        * math.exp(-28 / 125 - V / 125)
        / (1 + 66.5465065250986 * math.exp(0.14992503748125938 * V))
    ),
    beta_f = 0.0065 * math.exp(-3 / 5 - V / 50) / (1 + math.exp(-6 - V / 5)),
    f'(t) = (1 - f) * alpha_f - beta_f * f,

    # Expressions for the Time dependent outward current component
    i_x1 = (
        0.0019727757115328517
        * (-1 + 21.75840239619708 * math.exp(0.04 * V))
        * math.exp(-0.04 * V)
        * x1
    ),

    # Expressions for the X1 gate component
    alpha_x1 = (
        0.031158410986342627
        * math.exp(0.08264462809917356 * V)
        / (1 + 17.41170806332765 * math.exp(0.05714285714285714 * V))
    ),
    beta_x1 = (
        0.0003916464405623223
        * math.exp(-0.05998800239952009 * V)
        / (1 + math.exp(-4 / 5 - V / 25))
    ),
    x1'(t) = (1 - x1) * alpha_x1 - beta_x1 * x1,

    # Expressions for the Time independent outward current component
    i_K1 = 0.0035 * (4.6000000000000005 + 0.2 * V) / (
        1 - 0.39851904108451414 * math.exp(-0.04 * V)
    ) + 0.0035 * (-4 + 119.85640018958804 * math.exp(0.04 * V)) / (
        8.331137487687693 * math.exp(0.04 * V) + 69.4078518387552 * math.exp(0.08 * V)
    ),

    # Expressions for the Stimulus protocol component
    if (t - IstimStart - IstimPeriod * math.floor((t - IstimStart) / IstimPeriod) <= IstimPulseDuration)and(t <= IstimEnd)and(t >= IstimStart)
        Istim = IstimAmplitude
    else
        Istim = 0
    end,
    # Istim = (IstimAmplitude, if (t - IstimStart - IstimPeriod * math.floor((t - IstimStart) / IstimPeriod) <= IstimPulseDuration)and(t <= IstimEnd)and(t >= IstimStart) else 0,),
    # IstimAmplitude=0.5, IstimEnd=50000.0, IstimPeriod=1000.0,
    # IstimPulseDuration=1.0, IstimStart=10.0

    # Expressions for the Membrane component
    V'(t) = (-i_K1 - i_Na - i_s - i_x1 + Istim) / C,

    y1(t) = m(t),
    y2(t) = h(t),
    y3(t) = j(t),
    y4(t) = Cai(t),
    y5(t) = d(t),
    y6(t) = f(t),
    y7(t) = x1(t),
    y8(t) = V(t),
)

In [5]:
#v known, I_ext fixed
ode_fhn0 = @ODEmodel(
    v'(t) = v - v*v*v - w + (23/100),
    w'(t) = (v - a - b*w)/tau,
    y(t) = v(t)
)

v'(t) = -w(t) - v(t)^3 + v(t) + 23//100
w'(t) = (-a - w(t)*b + v(t))//tau
y(t) = v(t)


In [6]:
#v known, a fixed
ode_fhn1 = @ODEmodel(
    v'(t) = v - v*v*v - w + I_ext,
    w'(t) = (v + 3/10 - b*w)/tau,
    y(t) = v(t)
)

v'(t) = I_ext - w(t) - v(t)^3 + v(t)
w'(t) = (-w(t)*b + v(t) + 3//10)//tau
y(t) = v(t)


In [7]:
#v,w known, no parameters fixed
ode_fhn2 = @ODEmodel(
    v'(t) = v - v*v*v - w + I_ext,
    w'(t) = (v - a - b*w)/tau,
    y1(t) = v(t),
    y2(t) = w(t),
)

w'(t) = (-w(t)*b - a + v(t))//tau
v'(t) = I_ext - w(t) - v(t)^3 + v(t)
y1(t) = v(t)
y2(t) = w(t)


In [8]:
#w known, no parameters fixed
ode_fhn3 = @ODEmodel(
    v'(t) = v - v*v*v - w + I_ext,
    w'(t) = (v - a - b*w)/tau,
    y(t) = w(t),
)

v'(t) = I_ext - w(t) - v(t)^3 + v(t)
w'(t) = (-w(t)*b - a + v(t))//tau
y(t) = w(t)


In [9]:
print(assess_local_identifiability(ode_br))
print("\n")
print(assess_identifiability(ode_br))

Dict{Nemo.fmpq_mpoly, Bool}(v => 1, I_ext => 0, a => 0, b => 1, tau => 1, w => 0)
Dict{Any, Symbol}(I_ext => :nonidentifiable, a => :nonidentifiable, b => :globally, tau => :globally)

In [10]:
print(assess_local_identifiability(ode_fhn0))
print("\n")
print(assess_identifiability(ode_fhn0))

Dict{Nemo.fmpq_mpoly, Bool}(a => 1, b => 1, tau => 1, v => 1, w => 1)
Dict{Any, Symbol}(a => :globally, b => :globally, tau => :globally)

In [11]:
print(assess_local_identifiability(ode_fhn1))
print("\n")
print(assess_identifiability(ode_fhn1))

Dict{Nemo.fmpq_mpoly, Bool}(I_ext => 1, b => 1, tau => 1, v => 1, w => 1)
Dict{Any, Symbol}(I_ext => :globally, b => :globally, tau => :globally)

In [12]:
print(assess_local_identifiability(ode_fhn2))
print("\n")
print(assess_identifiability(ode_fhn2))

Dict{Nemo.fmpq_mpoly, Bool}(I_ext => 1, a => 1, b => 1, tau => 1, w => 1, v => 1)
Dict{Any, Symbol}(I_ext => :globally, a => :globally, b => :globally, tau => :globally)

In [13]:
print(assess_local_identifiability(ode_fhn3))
print("\n")
print(assess_identifiability(ode_fhn3))

Dict{Nemo.fmpq_mpoly, Bool}(v => 1, I_ext => 1, a => 1, b => 1, tau => 1, w => 1)
Dict{Any, Symbol}(I_ext => :globally, a => :globally, b => :globally, tau => :globally)